# EdgeNeXt

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elmadafri/the-wildfire-dataset/versions/1")

print("Path to dataset files:", path)

100%|██████████| 9.94G/9.94G [01:41<00:00, 105MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/1


In [ ]:
!rm /root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/1/the_wildfire_dataset/the_wildfire_dataset/val/fire/Both_smoke_and_fire/desktop.ini

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from timm import create_model

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

class CustomFireDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.binary_labels = []
        self.multi_class_labels = []

        # Define mappings for binary and multi-class labels
        binary_label_mapping = {'fire': 1, 'nofire': 0}
        multi_class_mapping = {
            'fire': {'Both_smoke_and_fire': 0, 'Smoke_from_fires': 1},
            'nofire': {'Fire_confounding_elements': 2, 'Forested_areas_without_confounding_elements': 3, 'Smoke_confounding_elements': 4}
        }

        # Traverse the root directory and collect image paths and labels
        for binary_label_name in os.listdir(root_dir):
            binary_label_path = os.path.join(root_dir, binary_label_name)
            if os.path.isdir(binary_label_path):
                # Assign binary label
                binary_label = binary_label_mapping[binary_label_name]

                # Traverse subclasses
                for subclass_name in os.listdir(binary_label_path):
                    subclass_path = os.path.join(binary_label_path, subclass_name)
                    if os.path.isdir(subclass_path):
                        # Assign multi-class label
                        multi_class_label = multi_class_mapping[binary_label_name][subclass_name]

                        # Collect all images in the subclass directory
                        for img_name in os.listdir(subclass_path):
                            img_path = os.path.join(subclass_path, img_name)
                            if os.path.isfile(img_path):
                                self.image_paths.append(img_path)
                                self.binary_labels.append(binary_label)
                                self.multi_class_labels.append(multi_class_label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        binary_label = self.binary_labels[idx]
        multi_class_label = self.multi_class_labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, (torch.tensor(binary_label, dtype=torch.float), torch.tensor(multi_class_label, dtype=torch.long))


In [ ]:
# Define transformations for the test set
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load your test dataset
test_dataset = CustomFireDataset(root_dir="/root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/1/the_wildfire_dataset/the_wildfire_dataset/test", transform=test_transforms)

# Create a DataLoader for the test dataset
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=10)

## Load Model

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from transformers import MobileViTForImageClassification, MobileViTImageProcessor

In [ ]:
# Use pretrained weights
model = MobileViTForImageClassification.from_pretrained("apple/mobilevit-small")

# Fire vs No Fire
in_features = model.classifier.in_features

# Strip out original classifier
model.classifier = nn.Identity()

# Binary Head
binary_head = nn.Sequential(
    nn.Linear(in_features, 1),
    nn.Sigmoid()
)

# Multi-class Head -> 5 output classes
multi_class_head = nn.Linear(in_features, 5)

In [ ]:
#print(model)

In [ ]:
model.load_state_dict(torch.load("mobileVit_fire_multi_classifier_Variant_A.pth"))

<ipython-input-14-50fb076fdb3b>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("mobileVit_fire_multi_classifier_Variant_A.pth"))


<All keys matched successfully>

In [ ]:
import torch.nn as nn
from torch.quantization import quantize_dynamic

model.eval()
#binary_head.eval()
#multi_class_head.eval()

quantized_model = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

# Quantize the binary head (IGNORE - ONLY QUANTIZE BACKBONE)
#binary_head = quantize_dynamic(binary_head, {nn.Linear}, dtype=torch.qint8)

# Quantize the multi-class head (IGNORE - ONLY QUANTIZE BACKBONE)
#multi_class_head = quantize_dynamic(multi_class_head, {nn.Linear}, dtype=torch.qint8)

## Test the Model

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import pandas as pd
import numpy as np
import torch

# Ensure quantized_model, binary_head, and multi_class_head are defined and loaded
# Make sure binary_head and multi_class_head are also on the CPU if you moved quantized_model to CPU
binary_head = binary_head.to('cpu')
multi_class_head = multi_class_head.to('cpu')

device = torch.device('cpu')
quantized_model = quantized_model.to(device)
quantized_model.eval()
binary_head.eval()
multi_class_head.eval()

# Initialize lists to store results
true_binary_labels = []
predicted_binary_probs = []  # Probabilities for the positive class (fire)

true_multi_class_labels = []
predicted_multi_class_labels = []

with torch.no_grad():
    for images, (binary_labels, multi_class_labels) in test_loader:
        images = images.to(device)
        binary_labels = binary_labels.to(device)
        multi_class_labels = multi_class_labels.to(device)

        # quantized_model returns ImageClassifierOutputWithNoAttention
        outputs = quantized_model(images)  # This is not a Tensor, but an output object.

        # Extract the logits as the features tensor
        features = outputs.logits  # logits is a Tensor

        # Now pass features to your heads (which are quantized linear layers expecting Tensors)
        binary_output = binary_head(features)
        multi_class_output = multi_class_head(features)

        # Convert binary logits to probabilities
        binary_probs = torch.sigmoid(binary_output).squeeze().cpu().numpy()

        # Get predicted class labels for multi-class classification
        _, predicted_multi_class = torch.max(multi_class_output, 1)
        predicted_multi_class = predicted_multi_class.cpu().numpy()

        # Store true labels and predictions as before
        true_binary_labels.extend(binary_labels.cpu().numpy())
        predicted_binary_probs.extend(binary_probs)
        true_multi_class_labels.extend(multi_class_labels.cpu().numpy())
        predicted_multi_class_labels.extend(predicted_multi_class)

# Combine all data into a single DataFrame
results = pd.DataFrame({
    "True Binary Labels": true_binary_labels,
    "Predicted Binary Probabilities": predicted_binary_probs,
    "True Multi-Class Labels": true_multi_class_labels,
    "Predicted Multi-Class Labels": predicted_multi_class_labels
})

# Save results to CSV
results.to_csv("quantized_mobilevit_test_results.csv", index=False)
#print("Test results saved to 'quantized_mobilevit_test_results.csv'.")

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
torch.save(quantized_model.state_dict(), "mobilevit_quantized_model.pth")

In [ ]:
import time  # For measuring time
import pandas as pd  # For saving to CSV
import torch
import torch.nn as nn

device = torch.device('cpu')
model = model.to(device)
binary_head = binary_head.to(device)

model.eval()
binary_head.eval()

prediction_times = []
true_binary_labels = []
predicted_binary_probs = []

with torch.no_grad():
    for images, (binary_labels, _) in test_loader:
        images = images.to(device)
        binary_labels = binary_labels.to(device)

        # Start timing the prediction
        start_time = time.time()

        # Forward pass through the non-quantized model to get features
        outputs = model(images)
        features = outputs.logits

        # Forward pass through binary head
        binary_output = binary_head(features)

        # End timing the prediction
        end_time = time.time()

        # Calculate the time taken for this prediction
        prediction_time = end_time - start_time
        prediction_times.append(prediction_time)

        # Convert binary logits to probabilities (already in sigmoid form from head)
        binary_probs = binary_output.squeeze().cpu().numpy()

        # Store the results
        true_binary_labels.extend(binary_labels.cpu().numpy())
        predicted_binary_probs.extend(binary_probs)

# Combine all data into a DataFrame
data = pd.DataFrame({
    "Prediction Times (seconds)": prediction_times
})

# Save to CSV
data.to_csv("cpu_mobilevit_prediction_time.csv", index=False)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
import time  # For measuring time
import pandas as pd  # For saving to CSV
import torch
import torch.nn as nn

# Ensure quantized_model and binary_head are defined and loaded
# and that they are on the CPU
binary_head = binary_head.to('cpu')
quantized_model = quantized_model.to('cpu')

quantized_model.eval()
binary_head.eval()

device = torch.device('cpu')

# Lists to store results
prediction_times = []
true_binary_labels = []
predicted_binary_probs = []

with torch.no_grad():
    for images, (binary_labels, _) in test_loader:
        images = images.to(device)
        binary_labels = binary_labels.to(device)

        # Start timing the prediction
        start_time = time.time()

        # Forward pass through the quantized model to get an output object
        outputs = quantized_model(images)
        features = outputs.logits

        # Pass features through the binary head
        binary_output = binary_head(features)

        # End timing the prediction
        end_time = time.time()

        # Calculate and store the time taken for this prediction
        prediction_time = end_time - start_time
        prediction_times.append(prediction_time)

        # Convert binary logits to probabilities
        binary_probs = torch.sigmoid(binary_output).squeeze().cpu().numpy()

        # Store true labels and predictions for binary classification
        true_binary_labels.extend(binary_labels.cpu().numpy())
        predicted_binary_probs.extend(binary_probs)

# Combine all data into a single DataFrame
data = pd.DataFrame({
    "Prediction Times (seconds)": prediction_times
})

# Save results to CSV
data.to_csv("quantized_mobilevit_prediction_times.csv", index=False)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
